# PyGPlates
In this notebook we will explore the use of Python as an interface for the paleomagnetism and plate-reconstruction software [GPlates](https://www.gplates.org/) using PyGPlates and [GPlately](https://github.com/GPlates/gplately).

This software provides paleomagnetists with an invaluable set of tools for modelling the movement of plates. For geologists PyGPlates enables the reconstruction of paleogeography, helping users visualize how continents and oceans have evolved over millions of years and allows understanding our samples in their geologic and geographic context of formation. Users can analyze geological data in the context of plate tectonics, understanding the spatial relationships between geological features and plate boundaries.

There are many advantages for using a programming interface for GPlates, rather than the GUI. Such as leveraging Python's extensive data analysis capabilities, advanced and more customisable visualisation and creation of workflows for batch processing.


First step is to install pygmt and gplately on our virtual machine

In [21]:
import warnings
warnings.filterwarnings("ignore")

!pip install -q condacolab
import condacolab
condacolab.install()
!mamba install pygmt
!mamba install gplately

✨🍰✨ Everything looks OK!

Looking for: ['pygmt']

[+] 0.0s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.1s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.2s
conda-forge/linux-64   3%
conda-forge/noarch    ⣾  [+] 0.3s
conda-forge/linux-64   8%
conda-forge/noarch    17%[+] 0.4s
conda-forge/linux-64  14%
conda-forge/noarch    30%[+] 0.5s
conda-forge/linux-64  14%
conda-forge/noarch    43%[+] 0.6s
conda-forge/linux-64  19%
conda-forge/noarch    57%[+] 0.7s
conda-forge/linux-64  25%
conda-forge/noarch    70%[+] 0.8s
conda-forge/linux-64  30%
conda-forge/noarch    70%[+] 0.9s
conda-forge/linux-64  30%
conda-forge/noarch    83%[+] 1.0s
conda-forge/linux-64  36%
conda-forge/noarch    83%[+] 1.1s
conda-forge/linux-64  36%
conda-forge/noarch    83%[+] 1.2s
conda-forge/linux-64  36%
conda-forge/noarch    97%conda-forge/noarch                                
[+] 1.3s
conda-forge/linux-64  41%[+] 1.4s
conda-forge/linux-64  47%[+] 1.5s
conda-forge/linux-64  58%[+] 1.6s
conda-fo

import the libraries

In [22]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import pygplates
import gplately
import pygmt


## GPlates Files
The classic approach using the older versions of PyGPlates would be to import files from a local directory. GPlately allows us to retrieve files hosted in a remote data server. Retrieving from the remote server are that you don't necessarily need to have the files stored anywhere and download them on the fly. However, you are limited to the files that are stored in the server. If you want to use alternate reconstructions you need to use local files.

The two type of GPlates files we'll be using are:


1. Rotation file (*.rot)

In the context of GPlates, a rotation file is a data file that contains information about the relative motion and rotation of tectonic plates over geological time. Rotation files are a crucial component of plate tectonic reconstructions and are used to model the movement of geological features (such as continents, ocean basins, and mountain ranges) through time. Rotation files will contain information about how different plates (identified with **plate IDs**) move relative to each other according to a **rotation pole** during a set **time interval**. The amount of rotation along the pole for the time interval is given by the **rotation angle**.


2. Gpml files (*.gpml)

A .gpml file, short for GPlates Markup Language file, is a specific file format used by GPlates, a plate tectonics software, for storing and exchanging geological and geophysical data. These files are written in XML (eXtensible Markup Language) format. The XML structure of a .gpml file is hierarchical, organizing data into nested elements and attributes. This structure allows for the representation of complex geological information. Elements in .gpml files can have attributes that provide additional information or metadata about the represented features. Metadata might include information about the age of a geological feature, the publication source, or the geological time period it represents.

In summary, gpml files serve as a standard data format for storing geological and geophysical information in the realm of plate tectonics research. They facilitate data exchange, analysis, and visualization within the GPlates software environment and promote interoperability among scientists working on understanding Earth's dynamic geological history. We can convert shape files into gpml files



In [ ]:
# Example of import using data server

# Call GPlately's DataServer from the download.py module
# gdownload = gplately.download.DataServer("Muller2019")

# Obtain all rotation files, topology features and static polygons from Muller et al. 2019
# rotation_model, topology_features, static_polygons = gdownload.get_plate_reconstruction_files()

For this workshop we'll use the plate reconstruction model of [Merdith et al. 2021](https://www.sciencedirect.com/science/article/pii/S0012825220305237)

In [ ]:
print(Input_dir+"1000_0_rotfile_Merdith_et_al.rot")
rot = Input_dir+"1000_0_rotfile_Merdith_et_al.rot"

https://raw.githubusercontent.com/pierosampaio/PythonWorkshop/main/Files/GPlates_files/1000_0_rotfile_Merdith_et_al.rot


In [25]:
rot_fname = "https://raw.githubusercontent.com/pierosampaio/PythonWorkshop/main/Files/GPlates_files/1000_0_rotfile_Merdith_et_al.rot"
rotation_model = pygplates.RotationModel(rot_fname)


OpenFileForReadingError: ignored

In [ ]:
Input_dir = "https://raw.githubusercontent.com/pierosampaio/PythonWorkshop/main/Files/GPlates_files/"

#rotation_model = pygplates.RotationModel( "https://raw.githubusercontent.com/pierosampaio/PythonWorkshop/main/Files/GPlates_files/1000_0_rotfile_Merdith_et_al.rot")
static_polygons = pygplates.FeatureCollection(Input_dir+"shapes_static_polygons_Merdith_et_al.gpml")
coastlines = pygplates.FeatureCollection(Input_dir+"shapes_coastlines_Merdith_et_al.gpml")
continents = pygplates.FeatureCollection(Input_dir+"shapes_continents_Merdith_et_al.gpml")

topology_filenames = [
    "1000-410-Convergence_Merdith_et_al.gpml",
    "1000-410-Divergence_Merdith_et_al.gpml",
    "1000-410-Topologies_Merdith_et_al.gpml",
    "1000-410-Transforms_Merdith_et_al.gpml",
    "410-250_plate_boundaries_Merdith_et_al.gpml",
    "250-0_plate_boundaries_Merdith_et_al.gpml",
    "TopologyBuildingBlocks_Merdith_et_al.gpml"
]

topology_features = pygplates.FeatureCollection()
for topology in topology_filenames:
  topology_feature.add(Input_dir+topology)


OpenFileForReadingError: ignored

In [ ]:
def plot_subduction(df,color="black",side="l"):

    fig.plot(df,
         pen=f"1p,{color}",
         style=f"f0.5c/0.15c+{side}+t",
         fill=color
        )


In [ ]:
model = gplately.PlateReconstruction(rotation_model, topology_features, static_polygons)

In [ ]:
time = 270 #Ma
subduction_data = model.tessellate_subduction_zones(time, ignore_warnings=True)
ridge_data = model.tessellate_mid_ocean_ridges(time, ignore_warnings=True)
gplot = gplately.PlotTopologies(model, coastlines=coast, time=time)

RT = gplot.get_ridges_and_transforms()
#SZ = gplot.get_trenches()
SZ_L = a = gplot.get_subduction_direction()[0]
SZ_R = a = gplot.get_subduction_direction()[1]
coastlines = gplot.get_coastlines()

In [ ]:
fig = pygmt.Figure()


fig.plot(
    RT.geometry,
    pen="1p,blue",
    region="d",
    projection="G15/-45/20c",
    frame=["afg",f"+t{time} Ma"]
)


plot_subduction(SZ_L)
plot_subduction(SZ_R,side="r")



fig.plot(
    coastlines,
    fill="gray",
    pen="thinnest"

)

fig.show()

# Working with Point Data

In [28]:
import sys
import os
os.listdir()

['.config', 'condacolab_install.log', 'sample_data']

In [26]:
CFB = pd.read_excel(r"https://raw.githubusercontent.com/pierosampaio/PythonWorkshop/main/"+
                     "Files/GPlates_files/All_Perm_CFB.xlsx")

CFB = gpd.GeoDataFrame(CFB,geometry=gpd.points_from_xy(CFB.LONGITUDE,CFB.LATITUDE))

In [ ]:
fig = pygmt.Figure()

fig.coast(
    region="d",
    projection="W25c",
    frame=["afg","+tPermian CFBs"],
    land="gray",
    shorelines="thinnest"
)

fig.plot(
    CFB,
    style="c0.3c",
    fill="red",
    pen="thinnest"
)

fig.show()

In [ ]:
gpts = gplately.Points(model,CFB.LONGITUDE,CFB.LATITUDE)

In [ ]:
rlons, rlats = gpts.reconstruct(270,return_array=True)
CFB["rlons"] = rlons
CFB["rlats"] = rlats

CFB_pb = CFB.loc[(CFB["PB206_PB204"].notnull()==True)
                 |(CFB["PB207_PB204"].notnull()==True)
                 |(CFB["PB208_PB204"].notnull()==True &(CFB["PB208_PB204"]<41))
                ]


def delta86(row):

    iso208=row["PB208_PB204"]
    iso206=row["PB206_PB204"]

    NHRL = (iso206*1.209)+15.627

    return (iso208-NHRL)*100


CFB_pb["delta86"] = CFB_pb.apply(delta86,axis=1)

In [ ]:
fig,ax = plt.subplots()

sns.scatterplot(
    data=CFB_pb,
    x="PB206_PB204",
    y="PB208_PB204",
    hue="Name",
    ax=ax
)

ax.plot(
    np.arange(17,21),
    (np.arange(17,21)*1.209)+15.627,
    ls="-",
    color="k"
)

ax.set(xlim=(16,21),ylim=(36,41))

In [ ]:
fig = pygmt.Figure()

pygmt.config(COLOR_FOREGROUND="lightskyblue1")

fig.plot(
    RT.geometry,
    pen="1p,black",
    region="d",
    projection="W25c",
    frame=["afg",f"+t{time} Ma"]
)


plot_subduction(SZ_L)
plot_subduction(SZ_R,side="r")



fig.plot(
    coastlines,
    fill="gray",
    pen="thinnest"

)

pygmt.makecpt(cmap="viridis",series=[CFB_pb.delta86.quantile(0.1),CFB_pb.delta86.quantile(0.9),10])


fig.plot(
    x=CFB_pb.rlons,y=CFB_pb.rlats,
    style="c0.4c",
    fill=CFB_pb.delta86,
    cmap=True,
    pen="thinnest"
)

fig.colorbar(frame=["a15"])


fig.show()